# Anomalous Financial Transaction Detection

본 대회의 과제는 금융 거래 데이터에서 **이상 거래를 탐지하는 기능**을 개선하고 활용도를 높이는 분류 AI모델을 개발하는 것입니다. 

특히, 클래스 불균형 문제를 해결하기 위해 오픈소스 생성형 AI 모델을 활용하여 부족한 클래스의 데이터를 보완하고, 이를 통해 분류 모델의 성능을 향상시키는 것이 핵심 목표입니다. 

이러한 접근을 통해 금융보안에 특화된 데이터 분석 및 활용 역량을 강화하여 전문 인력을 양성하고, 금융권의 AI 활용 어려움에 따른 해결 방안을 함께 모색하며 금융 산업의 AI 활용 활성화를 지원하는 것을 목표로 합니다.

# Import Library

In [2]:
# 제출 파일 생성 관련
import os
import zipfile

# 데이터 처리 및 분석
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm

# 머신러닝 전처리
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

# 머신러닝 모델
import xgboost as xgb

# 합성 데이터 생성
from sdv.metadata import SingleTableMetadata
from sdv.single_table import TVAESynthesizer


# To ignore all warnings
import warnings
warnings.filterwarnings('ignore')

# 생성 🏭

# Load Data

In [3]:
train_all = pd.read_csv("/workspace/Dataset/FSI/train.csv")
test_all = pd.read_csv("/workspace/Dataset/FSI/test.csv")

In [4]:
train = train_all.drop(columns="ID")

In [5]:
train["Fraud_Type"].value_counts()

m    118800
a       100
j       100
h       100
k       100
c       100
g       100
i       100
b       100
f       100
d       100
e       100
l       100
Name: Fraud_Type, dtype: int64

In [6]:
'''
(*) 리더보드 산식 중 생성데이터의 익명성(TCAP)채점을 위해 각 클래스 별로 1000개의 생성데이터가 반드시 필요합니다.
(*) 본 베이스 라인에서는 "Fraud_Type" 13종류에 대해 1000개씩 , 총 13,000개의 데이터를 생성할 예정입니다.
(*) 분류 모델 성능 개선을 위해 생성 데이터를 활용하는 것에는 생성 데이터의 Row 개수에 제한이 없습니다. 단, 리더보드 평가를 위해 제출을 하는 생성 데이터 프레임은 익명성(TCAP) 평가를 위함이며, 위의 조건을 갖춘 생성 데이터를 제출해야합니다.
'''
N_CLS_PER_GEN = 1000


In [7]:
'''
handle outlier generate 전에 안하기

'''

'\nhandle outlier generate 전에 안하기\n\n'

In [8]:

# 이상치 처리 함수
def handle_outliers(series, n_std=3):
    mean = series.mean()
    std = series.std()
    z_scores = np.abs(stats.zscore(series))
    return series.mask(z_scores > n_std, mean)

# Time_difference 컬럼을 총 초로 변환 및 이상치 처리
train['Time_difference_seconds'] = pd.to_timedelta(train['Time_difference']).dt.total_seconds()
# train['Time_difference_seconds'] = handle_outliers(train['Time_difference_seconds'])


# 모든 Fraud_Type 목록 생성 (m 포함)
fraud_types = train['Fraud_Type'].unique()

# 모든 합성 데이터를 저장할 DataFrame 초기화
all_synthetic_data = pd.DataFrame()

N_SAMPLE = 100

# 각 Fraud_Type에 대해 합성 데이터 생성 및 저장
for fraud_type in tqdm(fraud_types):
    
    # 해당 Fraud_Type에 대한 서브셋 생성
    subset = train[train["Fraud_Type"] == fraud_type]

    #m은 샘플링 더 하기
    if fraud_type == "m":
        N_SAMPLE = 10000
    else:
        N_SAMPLE = 100

    # 모든 Fraud_Type에 대해 100개씩 샘플링
    subset = subset.sample(n=N_SAMPLE, random_state=77)
    
    # Time_difference 열 제외 (초 단위로 변환된 컬럼만 사용)
    subset = subset.drop('Time_difference', axis=1)
    
    # 메타데이터 생성 및 모델 학습
    metadata = SingleTableMetadata()

    metadata.detect_from_dataframe(subset)
    metadata.set_primary_key(None)

    # 데이터 타입 설정
    column_sdtypes = {
        'Account_initial_balance': 'numerical',
        'Account_balance': 'numerical',
        'Customer_identification_number': 'categorical',  
        'Customer_personal_identifier': 'categorical',
        'Account_account_number': 'categorical',
        'IP_Address': 'ipv4_address',  
        'Location': 'categorical',
        'Recipient_Account_Number': 'categorical',
        'Fraud_Type': 'categorical',
        'Time_difference_seconds': 'numerical',
        'Customer_Birthyear': 'numerical'
    }

    # 각 컬럼에 대해 데이터 타입 설정
    for column, sdtype in column_sdtypes.items():
        metadata.update_column(
            column_name=column,
            sdtype=sdtype
        )
        
    synthesizer = TVAESynthesizer(
        metadata,
        epochs=500,
        batch_size=64,
        embedding_dim=256,
        compress_dims=(256, 128, 64),
        decompress_dims=(64, 128, 256),
        l2scale=1e-4,
        loss_factor=2.5
    )

    synthesizer.fit(subset)

    synthetic_subset = synthesizer.sample(num_rows=N_CLS_PER_GEN)
    
    # 생성된 Time_difference_seconds의 이상치 처리
    synthetic_subset['Time_difference_seconds'] = handle_outliers(synthetic_subset['Time_difference_seconds'])
    
    # Time_difference_seconds를 다시 timedelta로 변환
    synthetic_subset['Time_difference'] = pd.to_timedelta(synthetic_subset['Time_difference_seconds'], unit='s')
    
    # Time_difference_seconds 컬럼 제거
    synthetic_subset = synthetic_subset.drop('Time_difference_seconds', axis=1)
    
    # 생성된 데이터를 all_synthetic_data에 추가
    all_synthetic_data = pd.concat([all_synthetic_data, synthetic_subset], ignore_index=True)
# 최종 결과 확인
print("\nFinal All Synthetic Data Shape:", all_synthetic_data.shape)

100%|██████████| 13/13 [26:07<00:00, 120.61s/it]  


Final All Synthetic Data Shape: (13000, 63)


In [1]:
from sdv.evaluation.single_table import evaluate_quality

train_eval = train_all.drop(columns="ID")
# 메타데이터 생성 및 모델 학습
metadata = SingleTableMetadata()

metadata.detect_from_dataframe(train_eval)
metadata.set_primary_key(None)

# 데이터 타입 설정
column_sdtypes = {
    'Account_initial_balance': 'numerical',
    'Account_balance': 'numerical',
    'Customer_identification_number': 'categorical',  
    'Customer_personal_identifier': 'categorical',
    'Account_account_number': 'categorical',
    'IP_Address': 'ipv4_address',  
    'Location': 'categorical',
    'Recipient_Account_Number': 'categorical',
    'Fraud_Type': 'categorical',
    'Time_difference': 'numerical',
    'Customer_Birthyear': 'numerical'
}

# 각 컬럼에 대해 데이터 타입 설정
for column, sdtype in column_sdtypes.items():
    metadata.update_column(
        column_name=column,
        sdtype=sdtype
    )
    
quality_report = evaluate_quality(
    real_data=train_eval,
    synthetic_data=all_synthetic_data,
    metadata=metadata)

NameError: name 'train_all' is not defined

In [ ]:
from sdv.evaluation.single_table import get_column_plot

fig = get_column_plot(
    real_data=train,
    synthetic_data=all_synthetic_data,
    metadata=metadata,
    column_name='fraud_type'
)
    
fig.show()

In [9]:
all_synthetic_data.head()

,Customer_Birthyear,Customer_Gender,Customer_personal_identifier,Customer_identification_number,Customer_registration_datetime,Customer_credit_rating,Customer_flag_change_of_authentication_1,Customer_flag_change_of_authentication_2,Customer_flag_change_of_authentication_3,Customer_flag_change_of_authentication_4,...,Last_bank_branch_transaction_datetime,Flag_deposit_more_than_tenMillion,Unused_account_status,Recipient_account_suspend_status,Number_of_transaction_with_the_account,Transaction_history_with_the_account,First_time_iOS_by_vulnerable_user,Fraud_Type,Transaction_resumed_date,Time_difference
0,1956,female,한은정,nUEocJ-FflsFfn,2006-10-18 00:02:11,B,1,1,1,1,...,2025-05-01 22:03:21,0,0,0,1,1,0,m,2030-12-18 09:03:10,2 days 09:47:46
1,1966,female,지영환,vivVZE-wJNHEko,2004-10-12 17:38:31,C,0,1,0,1,...,2031-11-26 16:58:56,0,0,1,0,2,0,m,2028-06-22 07:57:10,3 days 10:41:48
2,1960,male,허예준,NCbLmA-AlbenMw,2011-05-02 20:23:19,B,1,1,1,1,...,2003-07-13 17:48:25,0,1,0,0,0,0,m,2013-09-04 03:31:28,4 days 15:06:25
3,1971,male,김정호,TclubG-HUSfOJN,2004-10-31 09:25:47,B,1,1,1,1,...,2033-10-03 18:38:22,0,1,0,0,0,0,m,2041-08-18 11:19:12,3 days 05:03:29
4,1999,male,이상철,mXIbDf-cOexzTt,2012-04-06 12:05:11,B,1,1,1,1,...,2031-05-20 05:10:38,0,0,0,0,2,0,m,2032-01-01 19:47:46,-1 days +16:00:47


## 원본 데이터와 concat

In [10]:
origin_train = train_all.drop(columns="ID")
train_total = pd.concat([origin_train, all_synthetic_data])
train_total.shape

(133000, 63)

# Data Preprocessing 1 : Select x, y

In [11]:
train_x = train_total.drop(columns=['Fraud_Type'])
train_y = train_total['Fraud_Type']

test_x = test_all.drop(columns=['ID'])

# Data Preprocessing 2 : 범주형 변수 인코딩

In [12]:
le_subclass = LabelEncoder()
train_y_encoded = le_subclass.fit_transform(train_y)

# 변환된 레이블 확인
for i, label in enumerate(le_subclass.classes_):
    print(f"원래 레이블: {label}, 변환된 숫자: {i}")

원래 레이블: a, 변환된 숫자: 0
원래 레이블: b, 변환된 숫자: 1
원래 레이블: c, 변환된 숫자: 2
원래 레이블: d, 변환된 숫자: 3
원래 레이블: e, 변환된 숫자: 4
원래 레이블: f, 변환된 숫자: 5
원래 레이블: g, 변환된 숫자: 6
원래 레이블: h, 변환된 숫자: 7
원래 레이블: i, 변환된 숫자: 8
원래 레이블: j, 변환된 숫자: 9
원래 레이블: k, 변환된 숫자: 10
원래 레이블: l, 변환된 숫자: 11
원래 레이블: m, 변환된 숫자: 12


In [13]:
# train_x
# 'Time_difference' 열을 문자열로 변환
train_x['Time_difference'] = train_x['Time_difference'].astype(str)

# 범주형 변수 인코딩
categorical_columns = train_x.select_dtypes(include=['object', 'category']).columns
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

# 훈련 데이터 인코딩
train_x_encoded = train_x.copy()
train_x_encoded[categorical_columns] = ordinal_encoder.fit_transform(train_x[categorical_columns])

# 특성 순서 저장
feature_order = train_x_encoded.columns.tolist()

# Model Define

In [14]:
# 모델 정의 및 학습
model = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    use_label_encoder=False,
    eval_metric='mlogloss'
)
model.fit(train_x_encoded[feature_order], train_y_encoded)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [15]:
# 테스트 데이터 인코딩
test_x_encoded = test_x.copy()
test_x_encoded[categorical_columns] = ordinal_encoder.transform(test_x[categorical_columns])


# 특성 순서 맞추기 및 데이터 타입 일치
test_x_encoded = test_x_encoded[feature_order]
for col in feature_order:
    test_x_encoded[col] = test_x_encoded[col].astype(train_x_encoded[col].dtype)

In [16]:
# 예측
predictions = model.predict(test_x_encoded)
predictions_label = le_subclass.inverse_transform(predictions)

# Submission

In [17]:
# 분류 예측 결과 제출 데이터프레임(DataFrame)
# 분류 예측 결과 데이터프레임 파일명을 반드시 clf_submission.csv 로 지정해야합니다.
clf_submission = pd.read_csv("/workspace/Dataset/FSI/sample_submission.csv")
clf_submission["Fraud_Type"] = predictions_label
clf_submission.head()

,ID,Fraud_Type
0,TEST_000000,b
1,TEST_000001,m
2,TEST_000002,m
3,TEST_000003,m
4,TEST_000004,m


In [18]:
# 합성 데이터 생성 결과 제출 데이터프레임(DataFrame)
# 합성 데이터 생성 결과 데이터프레임 파일명을 반드시 syn_submission.csv 로 지정해야합니다.
all_synthetic_data.head()

,Customer_Birthyear,Customer_Gender,Customer_personal_identifier,Customer_identification_number,Customer_registration_datetime,Customer_credit_rating,Customer_flag_change_of_authentication_1,Customer_flag_change_of_authentication_2,Customer_flag_change_of_authentication_3,Customer_flag_change_of_authentication_4,...,Last_bank_branch_transaction_datetime,Flag_deposit_more_than_tenMillion,Unused_account_status,Recipient_account_suspend_status,Number_of_transaction_with_the_account,Transaction_history_with_the_account,First_time_iOS_by_vulnerable_user,Fraud_Type,Transaction_resumed_date,Time_difference
0,1956,female,한은정,nUEocJ-FflsFfn,2006-10-18 00:02:11,B,1,1,1,1,...,2025-05-01 22:03:21,0,0,0,1,1,0,m,2030-12-18 09:03:10,2 days 09:47:46
1,1966,female,지영환,vivVZE-wJNHEko,2004-10-12 17:38:31,C,0,1,0,1,...,2031-11-26 16:58:56,0,0,1,0,2,0,m,2028-06-22 07:57:10,3 days 10:41:48
2,1960,male,허예준,NCbLmA-AlbenMw,2011-05-02 20:23:19,B,1,1,1,1,...,2003-07-13 17:48:25,0,1,0,0,0,0,m,2013-09-04 03:31:28,4 days 15:06:25
3,1971,male,김정호,TclubG-HUSfOJN,2004-10-31 09:25:47,B,1,1,1,1,...,2033-10-03 18:38:22,0,1,0,0,0,0,m,2041-08-18 11:19:12,3 days 05:03:29
4,1999,male,이상철,mXIbDf-cOexzTt,2012-04-06 12:05:11,B,1,1,1,1,...,2031-05-20 05:10:38,0,0,0,0,2,0,m,2032-01-01 19:47:46,-1 days +16:00:47


In [19]:
'''
(*) 저장 시 각 파일명을 반드시 확인해주세요.
    1. 분류 예측 결과 데이터프레임 파일명 = clf_submission.csv
    2. 합성 데이터 생성 결과 데이터프레임 파일명 = syn_submission.csv

(*) 제출 파일(zip) 내에 두 개의 데이터프레임이 각각 위의 파일명으로 반드시 존재해야합니다.
(*) 파일명을 일치시키지 않으면 채점이 불가능합니다.
'''

# 폴더 생성 및 작업 디렉토리 변경
os.makedirs('/workspace/Dataset/FSI/baseline+tvae_finetune', exist_ok=True)
os.chdir("/workspace/Dataset/FSI/baseline+tvae_finetune")

# CSV 파일로 저장
clf_submission.to_csv('/workspace/Dataset/FSI/baseline+tvae_finetune/clf_submission.csv', encoding='UTF-8-sig', index=False)
all_synthetic_data.to_csv('/workspace/Dataset/FSI/baseline+tvae_finetune/syn_submission.csv', encoding='UTF-8-sig', index=False)

# ZIP 파일 생성 및 CSV 파일 추가
with zipfile.ZipFile("/workspace/Dataset/FSI/baseline+tvae_finetune/baseline_submission.zip", 'w') as submission:
    submission.write('clf_submission.csv')
    submission.write('syn_submission.csv')
    
print('Done.')

Done.
